In [1]:
#import dependencies
import pandas as pd, gmaps, requests

#import api K
from config import g_key

In [39]:
#import csv to df
city_df = pd.read_csv("weather_data/cities.csv")
#temp_shift = city_df["Max Temp"].min()
#city_df["Max Temp"] = city_df["Max Temp"].add(-temp_shift,fill_value=0)
city_df.loc[city_df["Max Temp"] < 0, "Max Temp"] = 0
city_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Chuy,UY,2020-02-22 23:36:37,-33.70,-53.46,60.1,86,15,7.02
1,1,Shache,CN,2020-02-22 23:50:57,38.42,77.24,37.0,39,0,4.99
2,2,Yellowknife,CA,2020-02-22 23:39:56,62.46,-114.35,3.2,65,75,6.93
3,3,Saint-Pierre,RE,2020-02-22 23:42:30,-21.34,55.48,75.2,69,85,8.05
4,4,Rio Gallegos,AR,2020-02-22 23:43:15,-51.62,-69.22,75.2,33,68,19.46


In [40]:
#configure gmaps to use API
gmaps.configure(api_key=g_key)
#heatmap of temp
#get lat long
locations = city_df[["Lat","Lng"]]
#get max temp
max_temp = city_df["Max Temp"]
#initialize a gmaps
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.2)
#assign heatmap
heat_layer = gmaps.heatmap_layer(locations,weights=max_temp,
                                dissipating=False, max_intensity=400,
                                point_radius=5)
#add heatmap
fig.add_layer(heat_layer)
#show figure
fig

Figure(layout=FigureLayout(height='420px'))

In [33]:
# Heatmap of percent humidity
locations = city_df[["Lat", "Lng"]]
humidity = city_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.2)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, 
                                 max_intensity=400, point_radius=5)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [34]:
# Heatmap of percent cloudiness
locations = city_df[["Lat", "Lng"]]
clouds = city_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.2)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, 
                                 dissipating=False, 
                                 max_intensity=400, point_radius=5)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [35]:
# Heatmap of percent cloudiness
locations = city_df[["Lat", "Lng"]]
wind = city_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.2)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, 
                                 dissipating=False, 
                                 max_intensity=400, point_radius=5)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [36]:
#ask customer to add min and max temp values
min_temp = float(input("What is the min temp you would like?"))
max_temp = float(input("What is the max temp you would like?"))

What is the min temp you would like?75
What is the max temp you would like?90


In [48]:
#create new df based on customer temp preferences
preferred_cities_df = city_df.loc[(city_df["Max Temp"] >= min_temp) & 
                          (city_df["Max Temp"] <= max_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
3,3,Saint-Pierre,RE,2020-02-22 23:42:30,-21.34,55.48,75.20,69,85,8.05
4,4,Rio Gallegos,AR,2020-02-22 23:43:15,-51.62,-69.22,75.20,33,68,19.46
8,8,Muisne,EC,2020-02-22 23:50:58,0.60,-80.03,80.60,94,40,5.82
10,10,Airai,TL,2020-02-22 23:41:41,-8.93,125.41,76.59,64,5,1.41
17,17,Hithadhoo,MV,2020-02-22 23:42:03,-0.60,73.08,82.53,80,85,14.88


In [51]:
#create hotel_df with less columns and new key
hotel_df = preferred_cities_df.copy()
hotel_df = hotel_df[["City","Country","Max Temp","Lat","Lng"]]
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
3,Saint-Pierre,RE,75.20,-21.34,55.48,
4,Rio Gallegos,AR,75.20,-51.62,-69.22,
8,Muisne,EC,80.60,0.60,-80.03,
10,Airai,TL,76.59,-8.93,125.41,
17,Hithadhoo,MV,82.53,-0.60,73.08,


In [59]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

#iterate through hotel df
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    
    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    try:
        # Grab the first hotel from the results and store the name.
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping")


Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping


In [63]:
#set info box per Gmaps
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [64]:
# Heatmap of temperature for vacation spots
locations = hotel_df[["Lat", "Lng"]]
temps = hotel_df["Max Temp"]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.2)
heat_layer = gmaps.heatmap_layer(locations, weights=temps, 
                                 dissipating=False, 
                                 max_intensity=300, point_radius=5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))